In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install pyspark


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=e9b8c75d049b985ca6673c4926c96190231b67eb0381bf2b8240980d39017836
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


### Create SparkSession:

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
.master ("local")\
.appName ("Colab")\
.config ('spark.ui.port', '4050')\
.getOrCreate ()


spark

### Read the DataFrames_sample.json file:

In [ ]:
df = spark.read.format('json') \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .load('drive/MyDrive/Spark/DataFrames_sample.json')

### Display part of the data and schema:


In [ ]:
df.show()

df.printSchema()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
|7.74|0.52|256GB SSD|  2|    MacBook| 8GB|       12"|11.04|  2.03|2016|
|8.94|0.68|128GB SSD|  3|MacBook Air| 8GB|     13.3"| 12.8|  2.96|2016|
| 8.0|20.3|  1TB SSD|  4|       iMac|64GB|       27"| 25.6|  20.8|2017|
+----+----+---------+---+-----------+----+----------+-----+------+----+

root
 |-- D: double (nullable = true)
 |-- H: double (nullable = true)
 |-- HDD: string (nullable = true)
 |-- Id: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- RAM: string (nullable = true)
 |-- ScreenSize: string (nullable = true)
 |-- W: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Year: long (nullable = true)



## Using SQL
### Create Temp View:

In [ ]:
df.createOrReplaceTempView('my_view')

result = spark.sql('SELECT * FROM my_view')

result.show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
|7.74|0.52|256GB SSD|  2|    MacBook| 8GB|       12"|11.04|  2.03|2016|
|8.94|0.68|128GB SSD|  3|MacBook Air| 8GB|     13.3"| 12.8|  2.96|2016|
| 8.0|20.3|  1TB SSD|  4|       iMac|64GB|       27"| 25.6|  20.8|2017|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Display "RAM"column and count "RAM" column:

In [ ]:
result = spark.sql('SELECT RAM FROM my_view')

result.show()

+----+
| RAM|
+----+
|16GB|
| 8GB|
| 8GB|
|64GB|
+----+



### Get all columns when "Year" column equal "2015"  

In [ ]:
result = spark.sql('SELECT * FROM my_view WHERE Year = 2015')

result.show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Get all when "Model" start with "M":

In [ ]:
result = spark.sql('SELECT Model FROM my_view')

result.show()

+-----------+
|      Model|
+-----------+
|MacBook Pro|
|    MacBook|
|MacBook Air|
|       iMac|
+-----------+



### Get all data when "Model" column equal "MacBook Pro"

In [ ]:
result = spark.sql('SELECT * FROM my_view WHERE Model = "MacBook Pro"')

result.show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Get all data with Multiple Conditions when "RAM" column equal "8GB" and "Model" column is "Macbook".

In [ ]:
result = spark.sql('SELECT * FROM my_view WHERE RAM = "8GB"')

result.show()

+----+----+---------+---+-----------+---+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model|RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+---+----------+-----+------+----+
|7.74|0.52|256GB SSD|  2|    MacBook|8GB|       12"|11.04|  2.03|2016|
|8.94|0.68|128GB SSD|  3|MacBook Air|8GB|     13.3"| 12.8|  2.96|2016|
+----+----+---------+---+-----------+---+----------+-----+------+----+



### Get all data with Multiple Conditions when "D" greater than or equal "8" and "Model" column is "iMac".

In [ ]:
result = spark.sql("SELECT * FROM my_view WHERE D >= 8 AND Model = 'iMac'")

result.show()

+---+----+-------+---+-----+----+----------+----+------+----+
|  D|   H|    HDD| Id|Model| RAM|ScreenSize|   W|Weight|Year|
+---+----+-------+---+-----+----+----------+----+------+----+
|8.0|20.3|1TB SSD|  4| iMac|64GB|       27"|25.6|  20.8|2017|
+---+----+-------+---+-----+----+----------+----+------+----+

